### Created July 31, 2025

In [1]:
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

In [16]:
def points_within_bounding_box(points, bbox):
    """
    Filters points within a bounding box.
    
    Args:
        points (list of tuples): List of (latitude, longitude) tuples.
        bbox (tuple): Bounding box as (min_lat, min_lon, max_lat, max_lon).
        
    Returns:
        list of tuples: Points within the bounding box.
    """
    min_lat, min_lon, max_lat, max_lon = bbox
    mask = [
        (lat, lon) for lat, lon in points
        if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon
    ]
    # inside_points = [
    #     (lat, lon) for lat, lon in points
    #     if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon
    # ]
    # return [i for i in range(len(lat)) if (lat[i], lon[i]) in inside_points]
    return mask

In [29]:
def get_cs2_sigma(ds, bin_start=0, bin_end=1024):
    sigma = ds.variables['sig0_1_20_ku'][:]  # Power waveforms for 20 Hz
    latitudes = ds.variables['lat_poca_20_ku'][:]  # Latitude (20 Hz)
    longitudes = ds.variables['lon_poca_20_ku'][:]  # Longitude (20 Hz)
    time = ds.variables['time_20_ku'][:]  # Ti


    n_to_plot = len(time)
    
    indices = range(n_to_plot)

    records = []

    for idx in indices:

        records.append({
            # 'index': idx,
            'latitude': latitudes[idx].values,
            'longitude': longitudes[idx].values,
            'time': time[idx].values,
            'sigma0': sigma[idx].values
        })

    df = pd.DataFrame.from_records(records)
    return df

In [76]:
choice = input("Enter 'thin' or 'thick': ").strip().lower()

if choice == "thin":
    # --- Thin code block ---
    print("Running thin code...")
    # THIN CASE
    # Load both subsetted datasets
    ds1 = xr.open_dataset("CS_OFFL_SIR_SIN_2__20240411T105309_20240411T105755_E001.nc")
    ds2 = xr.open_dataset("CS_OFFL_SIR_SIN_2__20240522T191214_20240522T191615_E001.nc")
    
    bbox = (80.23, -87.01, 80.30,  -86.70)
    # bbox = (80.237, -87.015, 80.299,  -86.794)
    min_lat, min_lon, max_lat, max_lon = bbox

    
    # --- Extract lat/lon arrays ---
    lat1 = ds1["lat_poca_20_ku"]
    lon1 = ds1["lon_poca_20_ku"] 
    lat2 = ds2["lat_poca_20_ku"]
    lon2 = ds2["lon_poca_20_ku"]
    
    # --- Build a boolean mask of the same shape ---
    mask1 = (
        (lat1 >= min_lat) & (lat1 <= max_lat) &
        (lon1 >= min_lon) & (lon1 <= max_lon)
    )
    mask2 = (
        (lat2 >= min_lat) & (lat2 <= max_lat) &
        (lon2 >= min_lon) & (lon2 <= max_lon)
    )
    
    # --- Apply mask to dataset ---
    ds1_bbox = ds1.where(mask1, drop=True)
    ds2_bbox = ds2.where(mask2, drop=True)

    ds1.close()
    ds2.close
    
    ### Jul 13, 2025
    # THIN
    ### Dropping by rec_count_20_ku, check that lat/lon start/end are correct
    # ds1_bbox_tmp = ds1.where(ds1.rec_count_20_ku>=ds1.rec_count_20_ku.values.min()+33, drop=True) # thick case 20240406 start 1 80.148 -97.050 end 32 80.066 -97.157
    # ds1_bbox = ds1_bbox_tmp.where(ds1_bbox_tmp.rec_count_20_ku<=ds1_bbox_tmp.rec_count_20_ku.values.min()+57-34, drop=True) # thick case 20240406 start 1 80.148 -97.050 end 32 80.066 -97.157

    # ds1_bbox = ds1.where(inside1, drop=True)

    # ds2_bbox = ds2.where(inside2, drop=True) #(ds2_tmp.rec_count_20_ku>=ds2.rec_count_20_ku.values.min()+36, drop=True) 
    print('done - thin')

elif choice == "thick":
    # --- Thick code block ---
    print("Running thick code...")
    ### Added July 13, 2025
    # THICK CASE
    
    # Load both subsetted datasets
    ds1 = xr.open_dataset("CS_OFFL_SIR_SIN_2__20240406T114803_20240406T115232_E001.nc")
    ds2 = xr.open_dataset("CS_OFFL_SIR_SIN_2__20240519T200408_20240519T200815_E001.nc")
      
    # Round lat/lon for intersection tolerance (adjust decimal as needed)
    
    bbox = (80.05, -97.9, 80.2,  -97.0)
    # bbox = (80.052, -97.577, 80.148,  -97.050)
    min_lat, min_lon, max_lat, max_lon = bbox

    
    # --- Extract lat/lon arrays ---
    lat1 = ds1["lat_poca_20_ku"]
    lon1 = ds1["lon_poca_20_ku"] 
    lat2 = ds2["lat_poca_20_ku"]
    lon2 = ds2["lon_poca_20_ku"]
    
    # --- Build a boolean mask of the same shape ---
    mask1 = (
        (lat1 >= min_lat) & (lat1 <= max_lat) &
        (lon1 >= min_lon) & (lon1 <= max_lon)
    )
    mask2 = (
        (lat2 >= min_lat) & (lat2 <= max_lat) &
        (lon2 >= min_lon) & (lon2 <= max_lon)
    )
    
    # --- Apply mask to dataset ---
    ds1_bbox = ds1.where(mask1, drop=True)
    ds2_bbox = ds2.where(mask2, drop=True)

    ds1.close()
    ds2.close()
    
    print('done - thick')
else:
    print("Invalid input. Please enter 'thin' or 'thick'.")


Enter 'thin' or 'thick':  thick


Running thick code...
done - thick


In [77]:
# print(ds1.info())

# ds1_bbox.info()
ds1_bbox['lat_poca_20_ku']
ds1_bbox['sig0_1_20_ku']
# ds1_bbox['lon_20_ku']
# ds2_bbox['lat_20_ku']
# ds1_bbox['time_20_ku']



<xarray.DataArray 'sig0_1_20_ku' (time_20_ku: 50)>
array([10.78     , 12.059999 , 11.9      , 11.91     , 12.29     ,
       11.809999 , 11.24     , 11.48     , 11.58     , 10.48     ,
       10.179999 , 10.65     , 10.679999 , 10.46     , 10.929999 ,
       10.92     , 11.07     , 10.849999 , 10.639999 , 10.679999 ,
       10.69     , 10.78     , 10.92     , 11.7699995, 11.63     ,
       11.58     , 11.46     , 11.889999 , 11.62     , 11.34     ,
       11.04     , 11.139999 , 11.219999 , 11.139999 , 11.16     ,
       10.94     , 11.059999 , 11.3      , 11.17     , 11.139999 ,
       11.179999 , 11.259999 , 11.58     , 11.719999 , 11.95     ,
       11.61     , 11.179999 , 11.42     , 11.809999 , 12.309999 ],
      dtype=float32)
Coordinates:
    lat_poca_20_ku  (time_20_ku) float64 80.19 80.18 80.17 ... 80.06 80.05 80.05
    lon_poca_20_ku  (time_20_ku) float64 -97.0 -97.0 -97.0 ... -97.17 -97.18
  * time_20_ku      (time_20_ku) datetime64[ns] 2024-04-06T11:49:01.173616 .....
Attributes:
    comment:        The measured backscatter from the surface, corrected for ...
    long_name:      backscatter coefficient (retracker 1)
    standard_name:  surface_backwards_scattering_coefficient_of_radar_wave
    units:          dB

In [49]:
plt.close('all')

In [74]:
sigma_0406 = get_cs2_sigma(ds1_bbox)
sigma_0406.to_csv('cs2_sigma_0406.csv', index=False)

In [79]:
sigma_0519 = get_cs2_sigma(ds2_bbox)
sigma_0519.to_csv('cs2_sigma_0519.csv', index=False)

In [63]:
sigma_0411 = get_cs2_sigma(ds1_bbox)
sigma_0411.to_csv('cs2_sigma_0411.csv', index=False)

In [64]:
sigma_0522 = get_cs2_sigma(ds2_bbox)
sigma_0522.to_csv('cs2_sigma_0522.csv', index=False)